In [1]:
import socket
import time
from datetime import datetime
from PIL import Image
import os, glob
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import MaxPooling2D
from keras.layers import Conv2D
from keras.layers import Activation, Dropout, Flatten, Dense
import numpy as np
import os

HOST = '192.168.137.242'##라즈베리 ip로 설정
PORT = 8585

# 분류 대상 카테고리 선택하기 
accident_dir = "C:/python3_project/image/train"
categories = ["Lilium","violet","other"]
nb_classes = len(categories)

# 이미지 크기 지정 
image_w = 64 
image_h = 64
pixels = image_w * image_h * 3

#
string_data = ''

###라즈베리파이 통신함수
def getFileFromServer(filename,string_data):
    data_transferred = 0
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock :
        sock.connect((HOST, PORT))
        sock.sendall(filename.encode())#파일이름 전송
        data = sock.recv(1024)#파일 수신 대기
        if not data:
            print('파일[%s]: 서버에 존재하지 않거나 전송중 오류발생' %filename)
            return
        with open('C:/python3_project/image/test/' + filename, 'wb') as f:
            try:
                while  data:
                    f.write(data)
                    data_transferred += len(data)
                    data = sock.recv(1024)
                    sock.sendall(string_data.encode())
            except Exception as e:
                print(e)
            finally :
                print('파일[%s] 및 환경값 전송종료. 전송량 [%d]' %(filename, data_transferred))

# 이미지 데이터 읽어 들이기 
X = []
Y = []
for idx, cat in enumerate(categories):
    # 레이블 지정 
    label = [0 for i in range(nb_classes)]
    label[idx] = 1
    # 이미지 
    image_dir = accident_dir + "/" + cat  ##제대로 들어가짐
    files = glob.glob(image_dir+"/*.jpg") ##
    for i, f in enumerate(files):
        img = Image.open(f) 
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)      # numpy 배열로 변환
        X.append(data)
        Y.append(label)
        #if i % 10 == 0:
            #print(i, "\n", data)
X = np.array(X)
Y = np.array(Y)
# 학습 전용 데이터와 테스트 전용 데이터 구분 
X_train, X_test, y_train, y_test = \
    train_test_split(X, Y)
xy = (X_train, X_test, y_train, y_test)

print('>>> data 저장중 ...')
np.save("C:/python3_project/image/7obj.npy", xy)
print("ok,", len(Y))

Using TensorFlow backend.


>>> data 저장중 ...
ok, 607


In [ ]:
while 1:
    #전송받은 사진으로 머신러닝 실행
    # 카테고리 지정하기
    categories = ["Lilium","violet","other"]
    nb_classes = len(categories)
    # 이미지 크기 지정하기
    image_w = 64
    image_h = 64
    # 데이터 열기
    X_train, X_test, y_train, y_test = np.load("C:/python3_project/image/7obj.npy")
    # 데이터 정규화하기(0~1사이로)
    X_train = X_train.astype("float") / 256
    X_test  = X_test.astype("float")  / 256
    print('X_train shape:', X_train.shape)

    # 모델 구조 정의 
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape = X_train.shape[1:], padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))

    model.add(Conv2D(64, (3, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # 전결합층
    model.add(Flatten())    # 벡터형태로 reshape
    model.add(Dense(512))   # 출력
    model.add(Activation('relu'))
    model.add(Dropout(0.9))

    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))
    
    # 모델 구축하기
    model.compile(loss='binary_crossentropy',   # 최적화 함수 지정
        optimizer='rmsprop',
        metrics=['accuracy'])
# 모델 확인
#    print(model.summary())
# 모델 훈련하기
#    hist=model.fit(X_train, y_train, batch_size=64, nb_epoch=10)
#    print(hist.history)
# 학습 완료된 모델 저장
    hdf5_file = "C:/python3_project/image/7obj-model.hdf5"
    if os.path.exists(hdf5_file):
        # 기존에 학습된 모델 불러들이기
        model.load_weights(hdf5_file)
    else:
        # 학습한 모델이 없으면 파일로 저장
        model.fit(X_train, y_train, batch_size=64, nb_epoch=10)
        model.save_weights(hdf5_file)
    # 적용해볼 이미지 
    test_image = 'C:/python3_project/image/test/test.jpg'
    # 이미지 resize
    img = Image.open(test_image)
    img = img.convert("RGB")
    img = img.resize((64,64))
    data = np.asarray(img)
    X = np.array(data)
    X = X.astype("float") / 256
    X = X.reshape(-1, 64, 64,3)
    # 예측
    pred = model.predict(X)  
    result = [np.argmax(value) for value in pred]   # 예측 값중 가장 높은 클래스 반환
    print( pred )
    print('New data category : ',categories[roooooooooooooooooooooooooooooooollloesult[0]])
    #   실행 결과를 통해 알게된 작물의 환경값을 아래 소켓에 전송
## 출력된 작물 이름을 통해 해당 작물의 환경값 라즈베리로 전송
    filename = 'test.jpg'
    if(categories[result[0]]=='Lilium') :
        string_data = '17.30.300.300.'
        getFileFromServer(filename,string_data)
        print('Lilium complete')
    elif(categories[result[0]]=='violet') :
        string_data = '10.40.400.300.'
        getFileFromServer(filename,string_data)
        print('violet complete')
    else :
        string_data = '20.40.400.350.'
        getFileFromServer(filename,string_data)
        print('other complete')
    time.sleep(12)
###(categories[result[0]]=='Banana')